In [1]:
import pandas as pd

In [2]:
test_data = pd.read_csv('../input/train.tsv', sep='\t')
#print(test_data)
#pd.read_csv('data/train.tsv', sep='\t')
testdf = test_data.values

Xtrain = testdf[:,2] #This will have all rows with index 2 col (Our reviews)
print(Xtrain)

ytrain = testdf[:,3] #This will all rows with index 3 col (reviews label)
#print(ytrain)

['A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .'
 'A series of escapades demonstrating the adage that what is good for the goose'
 'A series' ... 'avuncular chortles' 'avuncular' 'chortles']


In [3]:
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding

Using TensorFlow backend.


In [4]:
t = Tokenizer() #Technique to convert text list to text list index

In [5]:
t.fit_on_texts(Xtrain) 

In [6]:
vocab_size = len(t.word_index) + 1
print(vocab_size)

15289


In [7]:
# integer encode the documents
encoded_docs = t.texts_to_sequences(Xtrain)
print(encoded_docs[0])

[2, 323, 3, 14150, 6028, 1, 6586, 9, 52, 8, 46, 13, 1, 2976, 8, 177, 46, 13, 1, 10913, 65, 3, 78, 668, 10117, 19, 576, 3, 78, 2123, 5, 57, 3, 2, 42]


In [8]:
item = max(Xtrain, key=len)
print(len(item))

283


In [9]:
# pad documents to a max length of 4 words
max_length = 20
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[ 177   46   13 ...    3    2   42]
 [   2  323    3 ...    0    0    0]
 [   2  323    0 ...    0    0    0]
 ...
 [9376 9377    0 ...    0    0    0]
 [9376    0    0 ...    0    0    0]
 [9377    0    0 ...    0    0    0]]


In [10]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('../input/glove.6B.100d.txt', encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [11]:
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [12]:
# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 100))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [13]:
# define model
#model = Sequential()
#e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=4, trainable=False)
#model.add(e)
#model.add(Flatten())
#model.add(Dense(1, activation='sigmoid'))
# compile the model
#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
#print(model.summary())

model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=20, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(32, input_dim=20, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(5, activation='softmax'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [14]:
print(ytrain)

[1 2 2 ... 3 2 2]


In [15]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(ytrain)
encoded_Y = encoder.transform(ytrain)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

In [16]:
# fit the model
model.fit(padded_docs, dummy_y, epochs=500, verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/500
156060/156060 [==============================] - 15s 93us/step - loss: 1.0294 - acc: 0.5763
Epoch 2/500
156060/156060 [==============================] - 14s 92us/step - loss: 0.9573 - acc: 0.6020
Epoch 3/500
156060/156060 [==============================] - 14s 93us/step - loss: 0.9295 - acc: 0.6150
Epoch 4/500
156060/156060 [==============================] - 15s 96us/step - loss: 0.9075 - acc: 0.6233
Epoch 5/500
156060/156060 [==============================] - 14s 92us/step - loss: 0.8883 - acc: 0.6324
Epoch 6/500
156060/156060 [==============================] - 14s 90us/step - loss: 0.8725 - acc: 0.6367
Epoch 7/500
156060/156060 [==============================] - 14s 89us/step - loss: 0.8582 - acc: 0.6442
Epoch 8/500
156060/156060 [==============================] - 14s 90us/step - loss: 0.8456 - acc: 0.6490
Epoch 9/500
156060/156060 [==============================] - 14s 89us/step - loss: 0.8341 - acc: 0.6540
Epoch 10/500
156

156060/156060 [==============================] - 14s 87us/step - loss: 0.6531 - acc: 0.7321
Epoch 77/500
156060/156060 [==============================] - 14s 87us/step - loss: 0.6529 - acc: 0.7326
Epoch 78/500
156060/156060 [==============================] - 15s 98us/step - loss: 0.6515 - acc: 0.7345
Epoch 79/500
156060/156060 [==============================] - 14s 87us/step - loss: 0.6506 - acc: 0.7342
Epoch 80/500
156060/156060 [==============================] - 14s 88us/step - loss: 0.6499 - acc: 0.7341
Epoch 81/500
156060/156060 [==============================] - 14s 88us/step - loss: 0.6491 - acc: 0.7351
Epoch 82/500
156060/156060 [==============================] - 14s 87us/step - loss: 0.6482 - acc: 0.7347
Epoch 83/500
156060/156060 [==============================] - 14s 87us/step - loss: 0.6478 - acc: 0.7354
Epoch 84/500
156060/156060 [==============================] - 15s 98us/step - loss: 0.6469 - acc: 0.7357
Epoch 85/500
156060/156060 [==============================] - 14s 87

156060/156060 [==============================] - 15s 94us/step - loss: 0.6112 - acc: 0.7512
Epoch 154/500
156060/156060 [==============================] - 14s 88us/step - loss: 0.6111 - acc: 0.7514
Epoch 155/500
156060/156060 [==============================] - 14s 88us/step - loss: 0.6104 - acc: 0.7515
Epoch 156/500
156060/156060 [==============================] - 14s 88us/step - loss: 0.6105 - acc: 0.7522
Epoch 157/500
156060/156060 [==============================] - 15s 95us/step - loss: 0.6096 - acc: 0.7517
Epoch 158/500
156060/156060 [==============================] - 14s 92us/step - loss: 0.6099 - acc: 0.7525
Epoch 159/500
156060/156060 [==============================] - 14s 88us/step - loss: 0.6095 - acc: 0.7536
Epoch 160/500
156060/156060 [==============================] - 14s 88us/step - loss: 0.6091 - acc: 0.7526
Epoch 161/500
156060/156060 [==============================] - 14s 89us/step - loss: 0.6085 - acc: 0.7530
Epoch 162/500
156060/156060 [==============================]

156060/156060 [==============================] - 14s 88us/step - loss: 0.5901 - acc: 0.7611
Epoch 231/500
156060/156060 [==============================] - 15s 98us/step - loss: 0.5903 - acc: 0.7610
Epoch 232/500
156060/156060 [==============================] - 14s 88us/step - loss: 0.5902 - acc: 0.7618
Epoch 233/500
156060/156060 [==============================] - 14s 89us/step - loss: 0.5906 - acc: 0.7608
Epoch 234/500
156060/156060 [==============================] - 14s 88us/step - loss: 0.5894 - acc: 0.7621
Epoch 235/500
156060/156060 [==============================] - 14s 88us/step - loss: 0.5895 - acc: 0.7618
Epoch 236/500
156060/156060 [==============================] - 14s 89us/step - loss: 0.5894 - acc: 0.7621
Epoch 237/500
156060/156060 [==============================] - 15s 98us/step - loss: 0.5893 - acc: 0.7621
Epoch 238/500
156060/156060 [==============================] - 14s 89us/step - loss: 0.5890 - acc: 0.7620
Epoch 239/500
156060/156060 [==============================]

156060/156060 [==============================] - 14s 89us/step - loss: 0.5771 - acc: 0.7668
Epoch 308/500
156060/156060 [==============================] - 14s 89us/step - loss: 0.5771 - acc: 0.7668
Epoch 309/500
156060/156060 [==============================] - 14s 89us/step - loss: 0.5777 - acc: 0.7671
Epoch 310/500
156060/156060 [==============================] - 15s 98us/step - loss: 0.5771 - acc: 0.7668
Epoch 311/500
156060/156060 [==============================] - 14s 89us/step - loss: 0.5768 - acc: 0.7673
Epoch 312/500
156060/156060 [==============================] - 14s 89us/step - loss: 0.5764 - acc: 0.7681
Epoch 313/500
156060/156060 [==============================] - 14s 90us/step - loss: 0.5764 - acc: 0.7674
Epoch 314/500
156060/156060 [==============================] - 14s 89us/step - loss: 0.5766 - acc: 0.7676
Epoch 315/500
156060/156060 [==============================] - 14s 90us/step - loss: 0.5761 - acc: 0.7678
Epoch 316/500
156060/156060 [==============================]

156060/156060 [==============================] - 14s 91us/step - loss: 0.5676 - acc: 0.7713
Epoch 385/500
156060/156060 [==============================] - 14s 89us/step - loss: 0.5682 - acc: 0.7710
Epoch 386/500
156060/156060 [==============================] - 14s 89us/step - loss: 0.5675 - acc: 0.7713
Epoch 387/500
156060/156060 [==============================] - 14s 89us/step - loss: 0.5681 - acc: 0.7720
Epoch 388/500
156060/156060 [==============================] - 14s 89us/step - loss: 0.5674 - acc: 0.7719
Epoch 389/500
156060/156060 [==============================] - 16s 103us/step - loss: 0.5679 - acc: 0.7715
Epoch 390/500
156060/156060 [==============================] - 14s 90us/step - loss: 0.5678 - acc: 0.7720
Epoch 391/500
156060/156060 [==============================] - 14s 89us/step - loss: 0.5673 - acc: 0.7709
Epoch 392/500
156060/156060 [==============================] - 16s 101us/step - loss: 0.5671 - acc: 0.7713
Epoch 393/500
156060/156060 [=============================

156060/156060 [==============================] - 14s 90us/step - loss: 0.5608 - acc: 0.7738
Epoch 462/500
156060/156060 [==============================] - 15s 99us/step - loss: 0.5607 - acc: 0.7745
Epoch 463/500
156060/156060 [==============================] - 14s 89us/step - loss: 0.5607 - acc: 0.7743
Epoch 464/500
156060/156060 [==============================] - 14s 89us/step - loss: 0.5604 - acc: 0.7746
Epoch 465/500
156060/156060 [==============================] - 14s 89us/step - loss: 0.5605 - acc: 0.7741
Epoch 466/500
156060/156060 [==============================] - 14s 89us/step - loss: 0.5605 - acc: 0.7740
Epoch 467/500
156060/156060 [==============================] - 15s 94us/step - loss: 0.5599 - acc: 0.7747
Epoch 468/500
156060/156060 [==============================] - 15s 93us/step - loss: 0.5605 - acc: 0.7747
Epoch 469/500
156060/156060 [==============================] - 14s 89us/step - loss: 0.5599 - acc: 0.7739
Epoch 470/500
156060/156060 [==============================]

In [17]:
from keras.models import load_model

In [18]:
model.save('sentiment.h5')

The sentiment labels are:

0 - negative, 1 - somewhat negative, 2 - neutral, 3 - somewhat positive, 4 - positive

In [22]:
# integer encode the documents
Xtest = ['bad movie ever in the history', 'awesome movie, very good', 'bad', 'what a awesome movie', 'movie was good']

test_docs = t.texts_to_sequences(Xtest)
#print(test_docs)

test_padded_docs = pad_sequences(test_docs, maxlen=max_length, padding='post')
#print(test_padded_docs)

a = model.predict(test_padded_docs)
#print(a)

for x in a:
    m = max(x)
    print(m, [i for i, j in enumerate(x) if j == m][0])

0.7007351 0
0.84591436 4
0.7226222 1
0.57258713 4
0.65770465 3


In [24]:
testing_data = pd.read_csv('../input/test.tsv', sep='\t')
testingdf = testing_data.values

XphraseID = testingdf[:,0]
Xtest = testingdf[:,2]
print(Xtest)
#print(XphraseID)

['An intermittently pleasing but mostly routine effort .'
 'An intermittently pleasing but mostly routine effort' 'An' ...
 'A long-winded ,' 'A long-winded' 'predictable scenario']


In [27]:
import csv
test_docs = t.texts_to_sequences(Xtest)
#print(test_docs)

test_padded_docs = pad_sequences(test_docs, maxlen=max_length, padding='post')
#print(test_padded_docs)

a = model.predict(test_padded_docs)
#print(a)

submission = open('../input/Submission.csv','w')
columnTitleRow = "PhraseId, Sentiment\n"
submission.write(columnTitleRow)

for counter, x in enumerate(a):
    m = max(x)
    print(XphraseID[counter], m, [i for i, j in enumerate(x) if j == m][0])
    #submission.write(str(XphraseID[counter])+','+str([i for i, j in enumerate(x) if j == m][0])+'\n')
    PhraseId = str(XphraseID[counter])
    Sentiment = str([i for i, j in enumerate(x) if j == m][0])
    row = PhraseId + "," + Sentiment + "\n"
    submission.write(row)
    
submission.close()    
    

OSError: [Errno 30] Read-only file system: '../input/Submission.csv'